# Using QDK to explore the Carbon Error Correction Code.

This notebook will show how you can use the QDK with the Neutral Atom Device to explore how the Carbon error correction code runs with and without noise.

First, we import the required packages and utilities, initialize a `NeutralAtomDevice` and initialize the Q# environment with the project from the current folder. This includes a Python implementation of the Carbon decoder that can turn the raw physical qubit measurement results into corrected, logical qubit measurement results.

In [ ]:
import decoder
from qdk import qsharp, code
from qdk.widgets import Histogram
from qdk.simulation import NeutralAtomDevice, NoiseConfig

# Get an instance of the NeutralAtomDevice machine
device = NeutralAtomDevice()

# Initialize Q# with the project and adaptive profile configured
qsharp.init(target_profile=qsharp.TargetProfile.Adaptive_RIF, project_root=".")

Next, this `%%qsharp` cell sets up the experiment we want to run, which is parametrized by the number of repetitions, which Pauli basis to prepare and measure with, and how many blocks of Carbon should be used.

In [ ]:
%%qsharp
import Std.Diagnostics.Fact;
import Utils.TransversalCNOT;
import C12;

operation PerformTeleportExperiment(ec_repetitions : Int, basis : Pauli, num_blocks : Int) : (Result[], (Result[], Result[])[], Result[])[] {
    Fact(num_blocks > 0, "need at least one block");
    let size = (12 + 16); // 12 data + 16 ancillas per block
    use qubits = Qubit[size * num_blocks];
    mutable results = [];
    for i in 0..(num_blocks - 1) {
        let logical_block = qubits[(0+(i*size))..(11+(i*size))];
        let ancillas = qubits[(12+(i*size))..(27+(i*size))];

        results += [C12_Mark2_1Q_Teleport(ec_repetitions, basis, logical_block, ancillas)];
    }
    return results;
}

operation C12_Mark2_1Q_Teleport(ec_repetitions : Int, basis : Pauli, logical_block : Qubit[], ancillas : Qubit[]) : (Result[], (Result[], Result[])[], Result[]) {
    Fact(basis == PauliZ or basis == PauliX or basis == PauliI, "only PauliZ and PauliX supported");
    Fact(Length(logical_block) == 12, "logical block must be 12 qubits");
    Fact(Length(ancillas) == 16, "need 16 ancillas");

    // Prepare in the requested basis
    mutable preselect = if basis == PauliX {
        C12.PrepareXX(logical_block, ancillas[...3])
    } else {
        C12.PrepareZZ(logical_block, ancillas[...3])
    };

    mutable syndromes = [];

    for _ in 1..(ec_repetitions) {
        // Sequential teleport on..
        // Prepare Z, Teleport X
        set preselect += C12.PrepareZZ(ancillas[...11], ancillas[12...]);
        TransversalCNOT(logical_block, ancillas[...11]);
        ApplyToEach(H, logical_block);
        let syndrome_x = MResetEachZ(logical_block);

        // Prepare X, Teleport Z
        set preselect += C12.PrepareXX(logical_block, ancillas[12...]);
        TransversalCNOT(logical_block, ancillas[...11]);
        let syndrome_z = MResetEachZ(ancillas[...11]);
        set syndromes += [(syndrome_x, syndrome_z)];
    }

    // Final measurement
    if basis == PauliX {
        ApplyToEach(H, logical_block);
    }
    let final = MResetEachZ(logical_block);

    (preselect, syndromes, final)
}

With the experiment set up, we first try running the Q# with the default, sparse state simulation and no noise. This `Histogram` should have one bar with all `Zero` results, indicating that the logical teleport was performed successfully for one Carbon block in all 100 shots.

In [ ]:
results = qsharp.run(code.PerformTeleportExperiment, 100, 1, qsharp.Pauli.Z, 1)
corrected_logical_results = decoder.decode_results(results, "Z")

# Use counter to make a simple histogram of the corrected logical results.
Histogram(map(str, corrected_logical_results))

Next, we compile the program, this time with two Carbon blocks. To visualize and test more significant programs, try increasing the number of Carbon blocks (the last property) to larger values.

In [ ]:
qir = qsharp.compile(code.PerformTeleportExperiment, 1, qsharp.Pauli.Z, 2)


We can use the Neutral Atom Device we previously initialized to trace the compiled program. This will decompose the gates in the QIR and then perform layout and scheduling for the device.

In [ ]:
device.trace(qir)

We can also use the device to perform Clifford simulation of the Carbon program. Clifford simulation can scale to multiple Carbon blocks and multiple rounds of error correction. Here again, a noiseless simulation should produce a single bar in the histogram, this time with a number of logical result pairs equal to the number of Carbon blocks compiled into the program.

In [ ]:
res = device.simulate(qir, shots=1000, type="clifford")
corrected_logical_results = decoder.decode_results(res, "Z")
Histogram(map(str, corrected_logical_results))

Error correction is more interesting with noise configured, so now we can use the `NoiseConfig` class to specify what noise we'll use. These default values produce interesting results, showing some shots rejected as "preselect" and others corrected by the Carbon decoder. If we filter out those "preselect" results, we can see that the majority of the accepted results have been corrected into the expected result.

In [ ]:
noise = NoiseConfig()

def set_depolarizing_error(table, p: float):
    """One of three equally likely Pauli errors occurs with probability `p`."""
    table.x = p / 3
    table.y = p / 3
    table.z = p / 3

# 1Q error
set_depolarizing_error(noise.sx, 0.00225)

# Error per CZ
noise.cz.iz = 0.0105 / 3
noise.cz.zi = 0.0105 / 3
noise.cz.zz = 0.0105 / 3

# Loss per CZ
noise.cz.loss = 0.0027

# Error for moved atoms
noise.mov.z = 1e-3

# Loss per moved atom
noise.mov.loss = 0.0005

# Idle error during moves
noise.idle.s_probability = 1e-4

# Idle error during MCM: TODO
# Idle loss during MCM: TODO

res = device.simulate(qir, shots=10000, noise=noise, type="clifford")
corrected_logical_results = decoder.decode_results(res, "Z")
display(Histogram(map(str, corrected_logical_results)))
display(Histogram(map(str, filter(lambda res: not any(r == 'PREselect' for r in res), corrected_logical_results))))

To make it easier to see the results, this further processes them to treat each Carbon block as separate rather than plot the joint behavior of the blocks. This again shows both the full results in the first histogram and the non-preselect results in the second histogram.

In [ ]:
flattened = [r for res in corrected_logical_results for r in res]
display(Histogram(map(str, flattened)))
display(Histogram(map(str, filter(lambda res: res != 'PREselect', flattened))))